In [1]:
import pandas as pd
import numpy as np

In [2]:
author_path = r"H:\data lake\entities\author.parquet"
organization_path = r"H:\data lake\entities\organization.parquet"
publication_path = r"H:\data lake\entities\publication.parquet"
publication_author_path = r"H:\data lake\entities\publication_author.parquet"

In [3]:
author_data = pd.read_parquet(author_path)
organization_data = pd.read_parquet(organization_path)
publication_data = pd.read_parquet(publication_path)
publication_author_data = pd.read_parquet(publication_author_path)

In [4]:
author_data["organization_id"] = author_data["organization_id"].fillna(-1).astype(int) - 1

In [5]:
output = pd.merge(
        author_data, organization_data,
        how="left",
        left_on="organization_id",
        right_index=True,
        suffixes=("", "_y"),
)
output = output[["name", "h_index", "name_y"]]
output.rename({"name_y": "organization_name", "scopus_id_y": "organization_scopus_id"}, axis="columns", inplace=True)

publication_num = publication_author_data.groupby(by="author_id", as_index=False).count()
publication_num["author_id"] = publication_num["author_id"] - 1
output = pd.merge(
        output, publication_num,
        how="left",
        left_index=True,
        right_on="author_id"
)
output.rename({"publication_id": "publication_num", "author_id": "id"}, axis="columns", inplace=True)
output["id"] = output["id"] + 1
output.dropna(subset="publication_num", inplace=True)
output["publication_num"] = output["publication_num"].astype(int)

In [8]:
output[output["organization_name"] == "Perm National Research Polytechnic University"].to_csv("nodes.csv", index=False)

In [9]:
output

,name,h_index,organization_name,id,publication_num
0.0,"Maev, Igor V.",14,A.I. Yevdokimov Moscow State University of Med...,1,279
1.0,"Pushkar, Dmitry Yu",15,A.I. Yevdokimov Moscow State University of Med...,2,244
2.0,"Gurevich, Konstantin G.",8,A.I. Yevdokimov Moscow State University of Med...,3,124
3.0,"Adamian, Leila V.",15,Russian Ministry of Health,4,292
4.0,"Krylov, V. V.",9,A.I. Yevdokimov Moscow State University of Med...,5,212
...,...,...,...,...,...
446655.0,"Gruzdeva, Lyudmila A.",0,Yugra State University,446842,1
446656.0,"Biryukova, O. N.",1,Yugra State University,446843,1
446657.0,"Nazarov, V. P.",0,Yugra State University,446844,1
446658.0,"Prontisheva, S. P.",0,Yugra State University,446845,1


In [36]:
temp = pd.read_csv("nodes.csv")
temp_list = temp["id"].values
temp2 = pd.read_csv("edges.csv")
temp2_target = temp2["target"].values
temp2_source = temp2["source"].values
temp2_list = np.unique(np.array([temp2_source, temp2_target]))

In [38]:
output[output["id"].isin(temp2_list)].to_csv("nodes.csv", index=False)

array([ 33052,  49673,  50052, ..., 210112, 210113, 210114], dtype=int64)